# 4. Reflection

## Overview

The Reflection pattern introduces self-evaluation capabilities to agent systems. This architecture enables agents to:

- **Analyze their own outputs** for quality and completeness
- **Make iterative improvements** by regenerating responses
- **Ensure requirements are met** before finalizing responses
- **Implement quality control** through automated review cycles

![Reflection Architecture](../docs/reflection.png)

### Key Components

1. **Generator Agent**: Creates initial responses to user queries
2. **Reflection Agent**: Evaluates the quality and completeness of responses
3. **Feedback Loop**: Allows for regeneration based on reflection results
4. **Quality Criteria**: Structured evaluation framework

This pattern is particularly useful for:
- **Complex reasoning tasks** requiring multiple iterations
- **Content creation** with quality standards
- **Customer service** where accuracy is critical
- **Educational applications** with learning feedback

## Implementation Steps

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from typing import Literal
from langgraph.types import Command
from langgraph.graph import END

In [ ]:
llm = AzureChatOpenAI(model="gpt-4.1-mini")

In [ ]:
def travel_advice(state) -> str:
    print("Node: Travel Advice")
    response = llm.invoke(input=state["messages"])

    return {
        "messages": [response]
    }

In [ ]:
from pydantic import BaseModel
class ReflectionOutput(BaseModel):
    """You reflect on the conversation and checks if input contains the requested topic."""
    topic: str
    contains_info: bool

In [ ]:
def reflection(state) -> Command[Literal["travel_advice", "__end__"]]:
    print("Node: Reflection")
    last_message = state["messages"][-1]

    structured_llm = llm.with_structured_output(ReflectionOutput)

    topic_check_list = ["sports", "cultural", "historical"]
    validations = []
    for topic in topic_check_list:
        response = structured_llm.invoke(input=[
            SystemMessage(content=f"You are a content expert, and validate if a messages contains information about topic \"{topic}\"?"),
            last_message
        ])
        validations.append(response)

    missing_topics = [response for response in validations if not response.contains_info]
    print(f"Missing topics: {[response.topic for response in missing_topics]}")

    if len(missing_topics) > 0:
        return Command(
            goto="travel_advice",
            update={"messages": [AIMessage(content="Extend output with information about following topics: " 
                                           + ", ".join([response.topic for response in missing_topics]))]}
        )
    else:
        return Command(
            goto=END
        )

In [ ]:
from langgraph.graph import StateGraph, START
from langgraph.graph import MessagesState

builder = StateGraph(MessagesState)

builder.add_node("travel_advice", travel_advice)
builder.add_node("reflection", reflection)

builder.add_edge(START, "travel_advice")
builder.add_edge("travel_advice", "reflection")

graph = builder.compile()

In [ ]:
from IPython.display import Image
Image(graph.get_graph().draw_mermaid_png())

In [ ]:
messages = [
    SystemMessage(
        content="You are a helpful assistant that gives travel advice based on the user's input."
    ),
    HumanMessage(
        content="Perth"
    )
]

In [ ]:
for msg in messages:
    msg.pretty_print()

async for event in graph.astream(input={"messages": messages}, stream_mode="updates"):
    for node in graph.nodes.keys():
        node_output = event.get(node, {})
        if node_output is not None:
            output_msgs = node_output.get("messages", [])
            for msg in output_msgs:
                msg.pretty_print()